# Import Modules

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import argparse
import glob
import sys 
import yaml 
import glob
import h5py 
import ray
import logging 
import json
import gc
import cv2
import time
import itertools

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
# import io_dict_to_hdf5 as ioh5
import xarray as xr
import scipy.linalg as linalg
import scipy.sparse as sparse

from tqdm.notebook import tqdm, trange
from matplotlib.backends.backend_pdf import PdfPages
from scipy import signal
from pathlib import Path
from scipy.optimize import minimize_scalar
from scipy.interpolate import interp1d
from scipy.ndimage import shift as imshift
from sklearn.model_selection import train_test_split, GroupShuffleSplit
from sklearn.pipeline import make_pipeline
from sklearn import linear_model as lm 
from scipy.stats import binned_statistic
from sklearn.utils import shuffle
from sklearn.metrics import r2_score, mean_poisson_deviance
from numpy.lib.stride_tricks import sliding_window_view

# import torch
# import torch.nn as nn
# import torch.optim as optim
# import torch.nn.functional as F
# from torchvision import transforms
import torchvision
from scipy.ndimage import uniform_filter1d 
# from torch.utils.data import Dataset, DataLoader, TensorDataset, Subset
# torch.backends.cudnn.benchmark = True
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

sys.path.append(str(Path('.').absolute().parent))
from utils import *
import io_dict_to_hdf5 as ioh5
from format_data import load_ephys_data_aligned

pd.set_option('display.max_rows', None)
FigPath = check_path(Path('~/Research/SensoryMotorPred_Data').expanduser(),'Figures/Decoding')

ray.init(
    ignore_reinit_error=True,
    logging_level=logging.ERROR,
)
print(f'Dashboard URL: http://{ray.get_dashboard_url()}')
print('Dashboard URL: http://localhost:{}'.format(ray.get_dashboard_url().split(':')[-1]))

Decode out spatial patches from neural activity. 

nonlinear combining of visual and movement gives more predictive power then linear combining. 

frames and movement at time t predict visual info at t+1 with linear comb and nonlinear comb. combining in nonlin way would work better. 

how to make linear or nonlinear combinations of inputs? input is pixel intensities, and movment params output pixel intesities at t+1. 

mult case: random projection at same dimentionality. Pixels * movement. 

h0 just visual predicts only visual. movement "adds" something here. 

# Gather Data

In [ ]:
def load_train_test(file_dict, save_dir, model_dt=.1, frac=.1, train_size=.7, do_shuffle=False, do_norm=False, free_move=True, has_imu=True, has_mouse=False,):
    ##### Load in preprocessed data #####
    data = load_ephys_data_aligned(file_dict, save_dir, model_dt=model_dt, free_move=free_move, has_imu=has_imu, has_mouse=has_mouse,)
    if free_move:
        ##### Find 'good' timepoints when mouse is active #####
        nan_idxs = []
        for key in data.keys():
            nan_idxs.append(np.where(np.isnan(data[key]))[0])
        good_idxs = np.ones(len(data['model_active']),dtype=bool)
        good_idxs[data['model_active']<.5] = False
        good_idxs[np.unique(np.hstack(nan_idxs))] = False
    else:
        good_idxs = np.where((np.abs(data['model_th'])<10) & (np.abs(data['model_phi'])<10))[0]
    
    data['raw_nsp'] = data['model_nsp'].copy()
    ##### return only active data #####
    for key in data.keys():
        if (key != 'model_nsp') & (key != 'model_active') & (key != 'unit_nums'):
            data[key] = data[key][good_idxs] # interp_nans(data[key]).astype(float)
        elif (key == 'model_nsp'):
            data[key] = data[key][good_idxs]
        elif (key == 'unit_nums'):
            pass
    gss = GroupShuffleSplit(n_splits=1, train_size=train_size, random_state=42)
    nT = data['model_nsp'].shape[0]
    groups = np.hstack([i*np.ones(int((frac*i)*nT) - int((frac*(i-1))*nT)) for i in range(1,int(1/frac)+1)])

    for train_idx, test_idx in gss.split(np.arange(len(data['model_nsp'])), groups=groups):
        print("TRAIN:", len(train_idx), "TEST:", len(test_idx))


    data['model_dth'] = np.diff(data['model_th'],append=0)
    data['model_dphi'] = np.diff(data['model_phi'],append=0)

    data['model_vid_sm'] = (data['model_vid_sm'] - np.mean(data['model_vid_sm'],axis=0))/np.nanstd(data['model_vid_sm'],axis=0)
    data['model_vid_sm'][np.isnan(data['model_vid_sm'])]=0
    if do_norm:
        data['model_th'] = (data['model_th'] - np.mean(data['model_th'],axis=0))/np.std(data['model_th'],axis=0) 
        data['model_phi'] = (data['model_phi'] - np.mean(data['model_phi'],axis=0))/np.std(data['model_phi'],axis=0) 
        if free_move:
            data['model_roll'] = (data['model_roll'] - np.mean(data['model_roll'],axis=0))/np.std(data['model_roll'],axis=0) 
            data['model_pitch'] = (data['model_pitch'] - np.mean(data['model_pitch'],axis=0))/np.std(data['model_pitch'],axis=0) 

    ##### Split Data by train/test #####
    data_train_test = {
        'train_vid': data['model_vid_sm'][train_idx],
        'test_vid': data['model_vid_sm'][test_idx],
        'train_nsp': shuffle(data['model_nsp'][train_idx],random_state=42) if do_shuffle else data['model_nsp'][train_idx],
        'test_nsp': shuffle(data['model_nsp'][test_idx],random_state=42) if do_shuffle else data['model_nsp'][test_idx],
        'train_th': data['model_th'][train_idx],
        'test_th': data['model_th'][test_idx],
        'train_phi': data['model_phi'][train_idx],
        'test_phi': data['model_phi'][test_idx],
        'train_roll': data['model_roll'][train_idx] if free_move else [],
        'test_roll': data['model_roll'][test_idx] if free_move else [],
        'train_pitch': data['model_pitch'][train_idx] if free_move else [],
        'test_pitch': data['model_pitch'][test_idx] if free_move else [],
        'train_t': data['model_t'][train_idx],
        'test_t': data['model_t'][test_idx],
        'train_dth': data['model_dth'][train_idx],
        'test_dth': data['model_dth'][test_idx],
        'train_dphi': data['model_dphi'][train_idx],
        'test_dphi': data['model_dphi'][test_idx],
        'train_gz': data['model_gz'][train_idx] if free_move else [],
        'test_gz': data['model_gz'][test_idx] if free_move else [],
    }

    d1 = data
    d1.update(data_train_test)
    return d1,train_idx,test_idx

In [ ]:
free_move = True
if free_move:
    stim_type = 'fm1'
else:
    stim_type = 'hf1_wn' # 'fm1' # 

data_dir  = Path('~/Goeppert/freely_moving_ephys/ephys_recordings/070921/J553RT').expanduser() / stim_type
save_dir  = check_path(Path('~/Research/SensoryMotorPred_Data/data/070921/J553RT/').expanduser(), stim_type)
FigPath = check_path(FigPath, stim_type)
save_dir,data_dir,FigPath
# with open(save_dir / 'file_dict.json','r') as fp:
#     file_dict = json.load(fp)

In [ ]:
file_dict = {'cell': 0,
 'drop_slow_frames': True,
 'ephys': list(data_dir.glob('*ephys_merge.json'))[0].as_posix(),
 'ephys_bin': list(data_dir.glob('*Ephys.bin'))[0].as_posix(),
 'eye': list(data_dir.glob('*REYE.nc'))[0].as_posix(),
 'imu': list(data_dir.glob('*imu.nc'))[0].as_posix() if stim_type=='fm1' else None,
 'mapping_json': '/home/seuss/Research/Github/FreelyMovingEphys/probes/channel_maps.json',
 'mp4': True,
 'name': '070921_J553RT_control_Rig2_'+stim_type,
 'probe_name': 'DB_P128-6',
 'save': data_dir.as_posix(),
 'speed': None,
 'stim_type': 'light',
 'top': list(data_dir.glob('*TOP1.nc'))[0].as_posix() if stim_type=='fm1' else None,
 'world': list(data_dir.glob('*world.nc'))[0].as_posix(),}

In [ ]:
model_dt = .05
data,train_idx,test_idx = load_train_test(file_dict, save_dir, model_dt=model_dt, do_shuffle=False, do_norm=False,free_move=free_move, has_imu=free_move, has_mouse=False)
locals().update(data)

# Testing Tuning Curves

In [ ]:
# Create Tuning curve for theta
def tuning_curve(model_nsp, var, model_dt = .05, N_bins=10, Nstds=3):
    var_range = np.linspace(np.nanmean(var)-Nstds*np.nanstd(var), np.nanmean(var)+Nstds*np.nanstd(var),N_bins)
    tuning = np.zeros((model_nsp.shape[-1],len(var_range)-1))
    tuning_std = np.zeros((model_nsp.shape[-1],len(var_range)-1))
    for n in range(model_nsp.shape[-1]):
        for j in range(len(var_range)-1):
            usePts = (var>=var_range[j]) & (var<var_range[j+1])
            tuning[n,j] = np.nanmean(model_nsp[usePts,n])/model_dt
            tuning_std[n,j] = (np.nanstd(model_nsp[usePts,n])/model_dt)/ np.sqrt(np.count_nonzero(usePts))
    return tuning, tuning_std, var_range[:-1]


In [ ]:
tuning, tuning_std, var_range = tuning_curve(test_nsp, test_th, N_bins=10, model_dt=model_dt)

In [ ]:
n = 21
fig, axs = plt.subplots(1,figsize=(7,5))
axs.errorbar(var_range,tuning[n], yerr=tuning_std[n])
axs.set_ylim(bottom=0)
axs.set_xlabel('Eye Phi')
axs.set_ylabel('Spikes/s')
axs.set_title('Neuron: {}'.format(n))
plt.tight_layout()
# fig.savefig(FigPath/'ExampleTuningCurve.png',bbox_inches='tight',transparent=False, facecolor='w')

# Decoding Models

In [ ]:
@ray.remote
def do_glm_fit_vis_skl(train_nsp, test_nsp, y_train, y_test, celln, model_type, lag_list, bin_length=40, model_dt=.1):
    
    ##### Format data #####
    nt_glm_lag = len(lag_list)
    
    # Shift spikes by -lag for GLM fits
    sps_train = train_nsp[:,celln] # np.roll(train_nsp[:,celln],-lag)
    sps_test = test_nsp[:,celln] # np.roll(test_nsp[:,celln],-lag)


    if model_type == 'elasticnetcv':
        model = lm.ElasticNetCV(l1_ratio=[.05, .01, .5, .7]) # lm.RidgeCV(alphas=np.arange(100,10000,1000))) #  #MultiOutputRegressor(lm.Ridge(),n_jobs=-1)) 
        model.fit(sps_train,y_train)
        sta_all = np.reshape(model.coef_,(nt_glm_lag,)+nks)
        sp_pred = model.predict(y_test)
    elif model_type == 'ridgecv':
        lambdas = 1024 * (2**np.arange(0,16))
        model = lm.RidgeCV(alphas=lambdas)
        model.fit(sps_train,y_train)
        sta_all = np.reshape(model.coef_,(nt_glm_lag,)+nks)
        sp_pred = model.predict(y_test)
    return cc_all, sta_all, sps_test, sp_pred, r2_all

In [ ]:
lag=150 # in ms
nt_glm_lag = 5
# minlag = int(-lag//(1000*model_dt)); maxlag=int((lag//(1000*model_dt))+1)
lag_list = np.array([-1,0,1,2,3]) #,np.arange(minlag,maxlag,np.floor((maxlag-minlag)/nt_glm_lag).astype(int))
nt_glm_lag = len(lag_list)
print(lag_list,1000*lag_list*model_dt)
do_shuffle=False
model_type = 'ridgecv'


In [ ]:
winds_train = sliding_window_view(train_vid,(windn,windn),axis=(1,2))[:,::skipn,::skipn]
winds_train = winds_train.reshape(winds_train.shape[0],-1,winds_train.shape[-2],winds_train.shape[-1])
winds_test = sliding_window_view(test_vid,(windn,windn),axis=(1,2))[:,::skipn,::skipn]
winds_test = winds_test.reshape(winds_test.shape[0],-1,winds_test.shape[-2],winds_test.shape[-1])

In [ ]:
lambdas = 1024 * (2**np.arange(0,16))
# model = lm.RidgeCV(alphas=lambdas)
model = lm.MultiTaskElasticNetCV(l1_ratio=[.05, .01, .5, .7]) # lm.RidgeCV(alphas=np.arange(100,10000,1000))) #  #MultiOutputRegressor(lm.Ridge(),n_jobs=-1)) 
model.fit(train_nsp,winds_train[:,0].reshape(winds_train.shape[0],-1))
# sta_all = np.reshape(model.coef_,(nt_glm_lag,)+nks)
sp_pred = model.predict(test_nsp)

In [ ]:
nt_glm_lag = 1
nks = np.shape(train_vid)[1:]; nk = nks[0]*nks[1]*nt_glm_lag
lambdas = 1024 * (2**np.arange(0,16))
model = lm.RidgeCV(alphas=lambdas,verbose=True, n_jobs=-1)
# model = lm.MultiTaskElasticNetCV(cv=2,l1_ratio=[.01,.05,.5],verbose=True,selection='random',n_jobs=-1) # lm.RidgeCV(alphas=np.arange(100,10000,1000))) #  #MultiOutputRegressor(lm.Ridge(),n_jobs=-1)) 
model.fit(train_nsp,train_vid.reshape(train_vid.shape[0],-1))
# sta_all = np.reshape(model.coef_,(nt_glm_lag,)+nks)
sp_pred = model.predict(test_nsp)
pred_train = model.predict(train_nsp).reshape(-1,nks[0],nks[1])
pred_test = sp_pred.reshape(sp_pred.shape[0],nks[0],nks[1])

In [ ]:
sf = 2
pred_test_norm = normimgs(pred_test)
pred_test_up = np.zeros((pred_test.shape[0],sf*pred_test.shape[1],sf*pred_test.shape[2]))
test_vid_norm = normimgs(test_vid)
test_vid_up = np.zeros((test_vid.shape[0],sf*test_vid.shape[1],sf*test_vid.shape[2]))
pred_train_norm = normimgs(pred_train)
pred_train_up = np.zeros((pred_train.shape[0],sf*pred_train.shape[1],sf*pred_train.shape[2]))
train_vid_norm = normimgs(train_vid)
train_vid_up = np.zeros((train_vid.shape[0],sf*train_vid.shape[1],sf*train_vid.shape[2]))
for n in range(pred_test.shape[0]):
    pred_test_up[n] = cv2.resize(pred_test_norm[n],(sf*pred_test.shape[2],sf*pred_test.shape[1]))
    test_vid_up[n] = cv2.resize(test_vid_norm[n],(sf*test_vid.shape[2],sf*test_vid.shape[1]))
    pred_train_up[n] = cv2.resize(pred_train_norm[n],(sf*pred_train.shape[2],sf*pred_train.shape[1]))
    train_vid_up[n] = cv2.resize(train_vid_norm[n],(sf*train_vid.shape[2],sf*train_vid.shape[1]))

cond = 'test'
if cond == 'train':
    tot_samps = np.stack((pred_train_up, train_vid_up))
else:
    tot_samps = np.stack((pred_test_up, test_vid_up))
tot_samps.shape

In [ ]:
t = 1040 #2000
dt = 10
fig, ax = plt.subplots(1,10,figsize=(30,3))

for n,t in enumerate(np.arange(t,t+dt)):
    ax[n].imshow(pred_test_up[t,:,:], cmap='gray')
    ax[n].axis('off')
plt.suptitle('Decoding Frames')
plt.tight_layout()
# fig.savefig(FigPath/'Decoding_Frame.png',facecolor='white', transparent=True)

fig2, ax2 = plt.subplots(1,10,figsize=(30,3))
for n,t in enumerate(np.arange(t,t+dt)):
    ax2[n].imshow(test_vid_up[t,:,:], cmap='gray')
    ax2[n].axis('off')
plt.suptitle('Actual Frames')
plt.tight_layout()
# fig2.savefig(FigPath/'Decoding_Actual_Frame.png',facecolor='white', transparent=True)

In [ ]:
t = 200 #2000
dt = 200
im_grid = torchvision.utils.make_grid(torch.from_numpy(pred_test[t:t+dt,np.newaxis,:,:]),nrow=10,normalize=False)[0]
im_grid2 = torchvision.utils.make_grid(torch.from_numpy(test_vid[t:t+dt,np.newaxis,:,:]),nrow=10,normalize=False)[0]
fig, axs = plt.subplots(1,2,figsize=(20,20))
axs[0].imshow(im_grid, cmap='gray')#.permute(1,2,0))
axs[0].set_title('Decoding Prediction')
axs[1].imshow(im_grid2, cmap='gray')#.permute(1,2,0))
axs[1].set_title('Actual Frame')
plt.tight_layout()
# fig.savefig(FigPath/'DecodedMontage_{}.png'.format(model_type),bbox_inches='tight',transparent=False, facecolor='w')

In [ ]:
model_coeff = model.coef_.T.reshape(train_nsp.shape[-1],train_vid.shape[1],train_vid.shape[2])

In [ ]:
plt.imshow(model_coeff[51],cmap='RdBu')
plt.colorbar()

In [ ]:
im_grid = torchvision.utils.make_grid(torch.from_numpy(model_coeff[:,np.newaxis]),nrow=10,normalize=True)[0]
fig, axs = plt.subplots(1,1,figsize=(10,10))
axs.imshow(im_grid, cmap='gray')#.permute(1,2,0))
axs.set_title('Decoding Coeff')
plt.tight_layout()
# fig.savefig(FigPath/'DecodingWeights_{}.png'.format(model_type),bbox_inches='tight',transparent=False, facecolor='w')

In [ ]:
nks = np.shape(train_vid)[1:]; nk = nks[0]*nks[1]*nt_glm_lag
vid_pred = sp_pred.reshape(sp_pred.shape[0],nks[0],nks[1])
t=20
plt.imshow(vid_pred[t])
plt.colorbar()

In [ ]:
for do_shuffle in [False]:
    # Load Data
    data,train_idx,test_idx = load_train_test(file_dict, save_dir, model_dt=model_dt, do_shuffle=do_shuffle, do_norm=True,free_move=free_move, has_imu=free_move, has_mouse=False)
    locals().update(data)
    windn = 5
    skipn = 3
    ##### Start GLM Parallel Processing #####
    start = time.time()
    nks = np.shape(train_vid)[1:]; nk = nks[0]*nks[1]*nt_glm_lag
    # Reshape data (video) into (T*n)xN array
    winds_train = sliding_window_view(train_vid,(windn,windn),axis=(1,2))[:,::skipn,::skipn]
    winds_train = winds_train.reshape(winds_train.shape[0],-1,winds_train.shape[-2],winds_train.shape[-1])
    winds_test = sliding_window_view(test_vid,(windn,windn),axis=(1,2))[:,::skipn,::skipn]
    winds_test = winds_test.reshape(winds_test.shape[0],-1,winds_test.shape[-2],winds_test.shape[-1])
#     y_train = train_vid.reshape(train_vid.shape[0],-1)
#     y_test = test_vid.reshape(test_vid.shape[0],-1) 
    
    # Put data into shared memory for parallization 
    train_nsp_r = ray.put(train_nsp)
    test_nsp_r = ray.put(test_nsp)
    train_data_r = ray.put(x_train)
    test_data_r = ray.put(x_test)
    result_ids = []
    # Loop over parameters appending process ids
    for celln in range(train_nsp.shape[1]):
        result_ids.append(do_glm_fit_vis_skl.remote(train_nsp_r, test_nsp_r, train_data_r, test_data_r, celln, model_type, lag_list, model_dt=model_dt))

    print('N_proc:', len(result_ids))
    results_p = ray.get(result_ids)
    print('GLM Add: ', time.time()-start)

    ##### Gather Data and Find Max CC Model #####
    mcc = np.stack([results_p[i][0] for i in range(len(results_p))])
    msta = np.stack([results_p[i][1] for i in range(len(results_p))])
    msp = np.stack([results_p[i][2] for i in range(len(results_p))])
    mpred = np.stack([results_p[i][3] for i in range(len(results_p))])
    mr2 = np.stack([results_p[i][4] for i in range(len(results_p))])

#     nt_glm_lag = len(lag_list)
#     GLM_Data = {'mcc': mcc,
#                 'msta': msta,
#                 'msp': msp,
#                 'mpred': mpred,
#                 'mr2':mr2,}
#     if do_shuffle:
#         ioh5.save(save_dir/'GLM_{}_Data_VisOnly_notsmooth_dt{:03d}_T{:02d}_shuffled.h5'.format(model_type,int(model_dt*1000), nt_glm_lag), GLM_Data)
#     else:
#         ioh5.save(save_dir/'GLM_{}_Data_VisOnly_notsmooth_dt{:03d}_T{:02d}.h5'.format(model_type,int(model_dt*1000), nt_glm_lag), GLM_Data)
        
#     del train_nsp_r, test_nsp_r, train_data_r, test_data_r, result_ids, results_p, mcc, msta, msp, mpred, mr2,
#     gc.collect()

In [ ]:
train_vid

In [ ]:
x=np.arange(5000).reshape(50,10,10)

In [ ]:
windn = 5
skipn = 3
winds = sliding_window_view(train_vid,(windn,windn),axis=(1,2))[:,::skipn,::skipn]

In [ ]:
winds.shape

# Predciting Future input

In [ ]:

# Initialize movement combinations
titles = np.array(['Theta','Phi','Roll','Pitch']) # 'dg_p','dg_n' 'roll','pitch'
titles_all = []
for n in range(1,len(titles)+1):
    perms = np.array(list(itertools.combinations(np.arange(len(titles)), n)))
    for ind in range(perms.shape[0]):
        titles_all.append('_'.join([t for t in titles[perms[ind]]]))

train_dgaze_p = train_dth + np.diff(train_gz,append=0)
train_dgaze_n = train_dth - np.diff(train_gz,append=0)
test_dgaze_p = test_dth + np.diff(test_gz,append=0)
test_dgaze_n = test_dth - np.diff(test_gz,append=0)
move_train = np.hstack((train_th[:,np.newaxis],train_phi[:,np.newaxis],train_roll[:,np.newaxis],train_pitch[:,np.newaxis]))#, train_dth[:,np.newaxis],train_dphi[:,np.newaxis]))
move_test = np.hstack((test_th[:,np.newaxis],test_phi[:,np.newaxis],test_roll[:,np.newaxis],test_pitch[:,np.newaxis])) #,test_dgaze_p[:,np.newaxis],test_dgaze_n[:,np.newaxis]))#
# move_test = np.hstack((test_th[:,np.newaxis],test_phi[:,np.newaxis],test_dgaze_p[:,np.newaxis],test_dgaze_n[:,np.newaxis]))# test_dth[:,np.newaxis],test_dphi[:,np.newaxis]))
model_move = np.hstack((model_th[:,np.newaxis],model_phi[:,np.newaxis],model_roll[:,np.newaxis],model_pitch[:,np.newaxis])) #,test_dgaze_p[:,np.newaxis],test_dgaze_n[:,np.newaxis]))#
model_move = model_move - np.mean(model_move,axis=0)
move_test = move_test - np.mean(move_test,axis=0)
move_train = move_train - np.mean(move_train,axis=0)


lag_list = [5]
Tvid = train_vid.shape[0]
rolled_vid = np.hstack([np.roll(model_vid_sm, nframes, axis=0) for nframes in lag_list]) # nt_glm_lag
x_train = train_vid.reshape(len(train_idx),-1)
x_test = test_vid.reshape(len(test_idx),-1)

y_train = rolled_vid[train_idx].reshape(len(train_idx),-1)
y_test = rolled_vid[test_idx].reshape(len(test_idx),-1)

npx = train_vid.shape[1]*train_vid.shape[2]

In [ ]:
lambdas = 1024 * (2**np.arange(0,16))
model = lm.RidgeCV(alphas=lambdas)
model.fit(x_train,y_train)
# sta_all = np.reshape(model.coef_,(nt_glm_lag,)+nks)
vid_pred = model.predict(x_test)
cc = np.corrcoef(y_test,vid_pred)[0,1]
r2 = model.score(vid_pred,y_test)


n=4;ind=0
# for n in range(1,len(titles)+1):
perms = np.array(list(itertools.combinations(np.arange(len(titles)), n)))
#     for ind in range(perms.shape[0]):
x_train = np.concatenate((x_train,move_train[:,perms[ind]]),axis=1)
x_test = np.concatenate((x_test,move_test[:,perms[ind]]),axis=1)

lambdas = 1024 * (2**np.arange(0,16))
model = lm.RidgeCV(alphas=lambdas)
model.fit(x_train,y_train)
# sta_all = np.reshape(model.coef_,(nt_glm_lag,)+nks)
vid_predm = model.predict(x_test)
ccm = np.corrcoef(y_test,vid_predm)[0,1]
r2m = r2_score(vid_predm[:,:npx], y_test)


In [ ]:
model.alpha_

In [ ]:
model.coef_[:,npx:]

In [ ]:
plt.imshow(model.coef_[:,-4].reshape(20,30))
plt.colorbar()

In [ ]:
plt.imshow(model.coef_[500,:npx].reshape(20,30))
plt.colorbar()

In [ ]:
vid_predm.shape

In [ ]:
model.score(x_test,y_test)

In [ ]:
fig,ax = plt.subplots(figsize=(10,5))
ax.plot(x_test[:100,100] ,label='x_test')
ax.plot(y_test[:100,100] ,label='y_test')
ax.plot(vid_pred[:100,100] ,label='vid_pred')
ax.plot(vid_predm[:100,100] ,label='vid_predm')
plt.legend()

In [ ]:
t = 100
fig,axs = plt.subplots(1,3,figsize=(30,10))
ax = axs[0]
ax.imshow(y_test[t].reshape(20,30))
ax = axs[1]
ax.imshow(y_test[t-1].reshape(20,30))
ax = axs[2]
ax.imshow(vid_pred[t-5].reshape(20,30))


# Linear Models

## Regression on Movement

In [ ]:
do_shuffle = False
data = load_train_test(file_dict, save_dir, model_dt=model_dt, do_shuffle=do_shuffle, do_norm=True, free_move=free_move, has_imu=free_move, has_mouse=False)
locals().update(data)
Y_train = np.hstack((train_th[:,np.newaxis],train_phi[:,np.newaxis],train_roll[:,np.newaxis],train_pitch[:,np.newaxis]))#, train_dth[:,np.newaxis],train_dphi[:,np.newaxis]))
Y_test = np.hstack((test_th[:,np.newaxis],test_phi[:,np.newaxis],test_roll[:,np.newaxis],test_pitch[:,np.newaxis])) #,test_dgaze_p[:,np.newaxis],test_dgaze_n[:,np.newaxis]))#


In [ ]:
model_type = 'ridgecv'
if model_type == 'elasticnetcv':
    model = make_pipeline(StandardScaler(), lm.ElasticNetCV()) # lm.RidgeCV(alphas=np.arange(100,10000,1000))) #  #MultiOutputRegressor(lm.Ridge(),n_jobs=-1)) 
elif model_type == 'ridgecv':
    model = make_pipeline(StandardScaler(), lm.RidgeCV())

model.fit(train_nsp, Y_train)
pred_train = model.predict(train_nsp)
pred_test = model.predict(test_nsp)
train_score = model.score(train_nsp,Y_train)
test_score = model.score(test_nsp, Y_test)
print('Train Score:', train_score, 'Test Score:', test_score)
# print(model['model_type'].coef_[22])
cc = np.corrcoef(pred_test,Y_test)
print('cc={:.02f}'.format(cc[0,1]))


In [ ]:
##### Flip test and train #####
# model2 = make_pipeline(StandardScaler(), lm.RidgeCV())
# model2.fit(test_nsp, Y_test)
# pred_train = model2.predict(test_nsp)
# pred_test = model2.predict(train_nsp)
# train_score = model2.score(test_nsp,Y_test)
# test_score = model2.score(train_nsp, Y_train)
# print('Train Score:', train_score, 'Test Score:', test_score)
# print(model2['ridgecv'].coef_[22])

In [ ]:
model[model_type].alpha_

In [ ]:
pred_test.shape

In [ ]:
modeln = 3
plt.plot(Y_test[:100,modeln])

plt.plot(pred_test[:100,modeln])


In [ ]:
Y_train.shape

In [ ]:
t = 0
dt = 10000
plt.plot(np.arange(t,t+dt),Y_train[t:t+dt])
plt.plot(np.arange(t,t+dt), pred_train[t:t+dt])

In [ ]:
t = 100
dt = 100
fig, axs = plt.subplots(1,figsize=(7,5))
cc = np.corrcoef(Y_test,pred_test)[0,1]
axs.plot(np.arange(t,t+dt)*model_dt,Y_test[t:t+dt], 'k', label='Ground Truth')
axs.plot(np.arange(t,t+dt)*model_dt,pred_test[t:t+dt], 'r', label='Prediction')
axs.set_title('CorrCoeff: {:.02f}'.format(cc))
axs.set_xlabel('Time (s)')
# axs.set_ylabel('Eye Phi Angle')
axs.legend()
plt.tight_layout()
# fig.savefig(FigPath/'LinearRegressionExample_phi.png',bbox_inches='tight',transparent=False, facecolor='w')

In [ ]:
np.corrcoef(Y_train,pred_train)

In [ ]:
np.corrcoef(Y_test,pred_test)

In [ ]:
plt.scatter(Y_test,pred_test, alpha=.1)

In [ ]:
# plt.plot(model['elasticnetcv'].coef_)
plt.plot(model['ridgecv'].coef_)


In [ ]:
pred_test.shape,Y_test.shape

# Autocorrelation of the th, vid 

In [ ]:
from scipy.ndimage import uniform_filter1d 

In [ ]:
np.corrcoef(test_th, test_nsp[:,22])

In [ ]:
xcorr_data = plt.xcorr(test_th, test_nsp[:,22], maxlags=100)
lags, xscore = xcorr_data[0], xcorr_data[1]

In [ ]:
lags[np.argmax(xscore)], xscore[np.argmax(xscore)],lags[np.argmin(xscore)], xscore[np.argmin(xscore)]

In [ ]:
plt.acorr(train_th, maxlags=100)

## Regression on Video

In [ ]:
train_vid, test_vid, train_nsp, test_nsp, train_th, test_th, train_phi, test_phi, train_roll, test_roll, train_pitch, test_pitch, train_t, test_t, train_dth, test_dth, train_dphi, test_dphi = \
train_test_split(model_vid_sm, model_nsp, model_th, model_phi, model_roll, model_pitch, model_t, model_dth, model_dphi, train_size=.6, shuffle=False, random_state=0)

In [ ]:
Y_train = train_vid.reshape(train_vid.shape[0],-1)#[:,10:11] # np.stack((train_roll, train_pitch),axis=1) # 
Y_test = test_vid.reshape(test_vid.shape[0],-1)#[:,10:11] # np.stack((test_roll, test_pitch),axis=1) # 

In [ ]:
@ray.remote
def multi_regression(train_nsp,Y_train,test_nsp,Y_test,idx,model_type):
    if model_type == 'elasticnetcv':
        model = lm.ElasticNetCV() # lm.RidgeCV(alphas=np.arange(100,10000,1000))) #  #MultiOutputRegressor(lm.Ridge(),n_jobs=-1)) 
    elif model_type == 'ridgecv':
        model = lm.RidgeCV(alphas=np.arange(100,10000,1000))
        
    # MultiTaskElasticNetCV(n_jobs=-1)) # RidgeCV()# MultiTaskLassoCV(n_jobs=-1) # RidgeCV() # LinearRegression(n_jobs=-1) #
    # register_ray()
    # with joblib.parallel_backend('ray'):
    model.fit(train_nsp, Y_train[:,idx])
    pred_train = model.predict(train_nsp)
    pred_test = model.predict(test_nsp)
    model_coeff = model.coef_
#     print('Train Score:', model.score(train_nsp,Y_train), 'Test Score:', model.score(test_nsp, Y_test))
    train_score = np.corrcoef(pred_train,Y_train[:,idx])[0,1]
    test_score = np.corrcoef(pred_test, Y_test[:,idx])[0,1]
    alphas = model.alpha_
    return pred_train, pred_test, train_score, test_score, model_coeff, alphas


In [ ]:
model_type = 'ridgecv'

start = time.time()
train_nsp_r = ray.put(train_nsp)
Y_train_r = ray.put(Y_train)
test_nsp_r = ray.put(test_nsp)
Y_test_r = ray.put(Y_test)
result_ids = []
[result_ids.append(multi_regression.remote(train_nsp_r,Y_train_r,test_nsp_r,Y_test_r,idx,model_type)) for idx in range(0, train_vid.shape[-1]*train_vid.shape[-2])]
results_p = ray.get(result_ids)
print('MultiReg Time: ', time.time() - start)

In [ ]:
pred_train = np.stack([results_p[i][0] for i in range(len(results_p))])
pred_test = np.stack([results_p[i][1] for i in range(len(results_p))])
train_scores = np.array([results_p[i][2] for i in range(len(results_p))])
test_scores = np.array([results_p[i][3] for i in range(len(results_p))])
model_coeff = np.array([results_p[i][4] for i in range(len(results_p))])
alphas = np.array([results_p[i][5] for i in range(len(results_p))])


In [ ]:
pred_train = pred_train.T.reshape(pred_train.shape[-1],train_vid.shape[1],train_vid.shape[2])
pred_test = pred_test.T.reshape(pred_test.shape[-1],test_vid.shape[1],test_vid.shape[2])
model_coeff = model_coeff.T.reshape(train_nsp.shape[-1],train_vid.shape[1],train_vid.shape[2])

In [ ]:
# ElasticNet_data = {
#                 'train_scores': train_scores,
#                 'test_scores': test_scores,
#                 'pred_train': pred_train,
#                 'pred_test': pred_test, }
# ioh5.save(save_dir/'ElasticNet_data.h5',ElasticNet_data)

# Ridge_data = ioh5.load(save_dir/'RidgeData.h5')
# locals().update(ElasticNet_data)

In [ ]:
# import plotly.express as px
# t = 200
# dt = 500
# comb = np.concatenate((pred_train[t:t+dt,np.newaxis,:,:], train_vid[t:t+dt,np.newaxis,:,:]),axis=1)

# fig = px.imshow(comb, animation_frame=0, facet_col=1, binary_string=False)
# fig.update_layout(width=1000,
#                   height=500,
#                  )
# fig.show()

Need to look at decoding weights and see if they resempble receptive fields?

In [ ]:
pred_train.shape

## Plotting Decoded Video

In [ ]:
import cv2
import torchvision
from scipy.ndimage import uniform_filter1d 

In [ ]:
FigPath

In [ ]:
t = 1130 #2000
dt = 10
fig, ax = plt.subplots(1,10,figsize=(30,3))

for n,t in enumerate(np.arange(t,t+dt)):
    ax[n].imshow(pred_test_up[t,:,:], cmap='gray')
    ax[n].axis('off')
plt.suptitle('Decoding Frames')
plt.tight_layout()
fig.savefig(FigPath/'Decoding_Frame.png',facecolor='white', transparent=True)

fig2, ax2 = plt.subplots(1,10,figsize=(30,3))
for n,t in enumerate(np.arange(t,t+dt)):
    ax2[n].imshow(test_vid_up[t,:,:], cmap='gray')
    ax2[n].axis('off')
plt.suptitle('Actual Frames')
plt.tight_layout()
fig2.savefig(FigPath/'Decoding_Actual_Frame.png',facecolor='white', transparent=True)

In [ ]:
t = 1040 #2000
dt = 100
im_grid = torchvision.utils.make_grid(torch.from_numpy(pred_test[t:t+dt,np.newaxis,:,:]),nrow=10,normalize=False)[0]
im_grid2 = torchvision.utils.make_grid(torch.from_numpy(test_vid[t:t+dt,np.newaxis,:,:]),nrow=10,normalize=False)[0]
fig, axs = plt.subplots(2,1,figsize=(20,10))
axs[0].imshow(im_grid, cmap='gray')#.permute(1,2,0))
axs[0].set_title('Decoding Prediction')
axs[1].imshow(im_grid2, cmap='gray')#.permute(1,2,0))
axs[1].set_title('Actual Frame')
plt.tight_layout()
# fig.savefig(FigPath/'DecodedMontage_{}.png'.format(model_type),bbox_inches='tight',transparent=False, facecolor='w')

In [ ]:
im_grid = torchvision.utils.make_grid(torch.from_numpy(model_coeff[:,np.newaxis]),nrow=10,normalize=False)[0]
fig, axs = plt.subplots(1,1,figsize=(10,10))
axs.imshow(im_grid, cmap='gray')#.permute(1,2,0))
axs.set_title('Decoding Coeff')
plt.tight_layout()
# fig.savefig(FigPath/'DecodingWeights_{}.png'.format(model_type),bbox_inches='tight',transparent=False, facecolor='w')

In [ ]:
sf = 2
pred_test_norm = normimgs(pred_test)
pred_test_up = np.zeros((pred_test.shape[0],sf*pred_test.shape[1],sf*pred_test.shape[2]))
test_vid_norm = normimgs(test_vid)
test_vid_up = np.zeros((test_vid.shape[0],sf*test_vid.shape[1],sf*test_vid.shape[2]))
pred_train_norm = normimgs(pred_train)
pred_train_up = np.zeros((pred_train.shape[0],sf*pred_train.shape[1],sf*pred_train.shape[2]))
train_vid_norm = normimgs(train_vid)
train_vid_up = np.zeros((train_vid.shape[0],sf*train_vid.shape[1],sf*train_vid.shape[2]))
for n in range(pred_test.shape[0]):
    pred_test_up[n] = cv2.resize(pred_test_norm[n],(sf*pred_test.shape[2],sf*pred_test.shape[1]))
    test_vid_up[n] = cv2.resize(test_vid_norm[n],(sf*test_vid.shape[2],sf*test_vid.shape[1]))
    pred_train_up[n] = cv2.resize(pred_train_norm[n],(sf*pred_train.shape[2],sf*pred_train.shape[1]))
    train_vid_up[n] = cv2.resize(train_vid_norm[n],(sf*train_vid.shape[2],sf*train_vid.shape[1]))

cond = 'test'
if cond == 'train':
    tot_samps = np.stack((pred_train_up, train_vid_up))
else:
    tot_samps = np.stack((pred_test_up, test_vid_up))
tot_samps.shape

In [ ]:
# # Example Frames Video
# t = 0
# dt = pred_test.shape[0]
# # comb = np.concatenate((normimgs(pred_test),normimgs(test_vid)),axis=2)
# comb = np.concatenate((pred_test_up,test_vid_up),axis=2).astype(np.uint8)
# # comb = (comb - np.min(comb,axis=(-1,-2))[:,np.newaxis,np.newaxis])/(np.max(comb,axis=(-1,-2))-np.min(comb,axis=(-1,-2)))[:,np.newaxis,np.newaxis]
# # comb = (comb*255).astype(np.uint8)

# FPS = 10
# out = cv2.VideoWriter(os.path.join(FigPath,'Frames_ExVid.avi'), cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), FPS, (comb.shape[-1], comb.shape[-2]),0)
            
# for fm in tqdm(range(comb.shape[0])):
#     out.write(comb[fm])
# out.release()

In [ ]:
# ###### Grab data of longest continuous sequence ######
# def func1(a,b):
#     # "Enclose" mask with sentients to catch shifts later on
#     mask = np.r_[False,a,False]

#     # Get the shifting indices
#     idx = np.flatnonzero(mask[1:] != mask[:-1])

#     s0,s1 = idx[::2], idx[1::2]
#     idx_b = np.r_[0,(s1-s0).cumsum()]
#     out = []
#     for (i,j,k,l) in zip(s0,s1-1,idx_b[:-1],idx_b[1:]):
#         out.append(((i, j), b[k:l]))
#     return out

# train_idxs,test_idxs = train_test_split(good_idxs,train_size=.6,random_state=0)

# out = func1(test_idxs,np.arange(test_idxs.shape[0]))

# max_seqn = 0
# for n in range(len(out)):
#     if len(out[n][1]) > max_seqn:
#         max_seq = np.arange(out[n][0][0],out[n][0][1])
#         max_seqn = len(out[n][1])

In [ ]:
win_size = 3
tot_samps2 = uniform_filter1d(tot_samps,win_size,axis=1)

In [ ]:
t = 500
dt = 100
plt.plot(tot_samps2[0,t:t+dt,5,10])
plt.plot(tot_samps[1,t:t+dt,5,10])
plt.plot(tot_samps2[1,t:t+dt,5,10])
plt.legend(['Pred','Actual','Actual_smoothed'])

In [ ]:
from matplotlib.animation import FuncAnimation, PillowWriter, FFMpegWriter
from matplotlib import colors
def init():
    for n in range(2):
        axs[n].axis('off')
    plt.tight_layout()

def update(t):
    for n in range(2):
        ims[n].set_data(tot_samps2[n,t])
    plt.draw()

In [ ]:
t = 0# max_seq[0]
lat_dims = 2
x,y = [],[]
fig, axs = plt.subplots(1,2,figsize=(8,4))   #8,16,figsize=(50,30)  
axs = axs.flatten()
ims = []
titles = ['Pred','Actual']
for n in range(2):
    ims.append(axs[n].imshow(tot_samps2[n,t],cmap='gray',norm=colors.Normalize()))
    axs[n].axis('off')
    axs[n].set_title('{}'.format(titles[n]))
plt.tight_layout()
# fig.savefig(os.path.join(FigurePath,'testimg.png'))

In [ ]:
# writervideo = PillowWriter(fps=60)  
ani = FuncAnimation(fig, update, tqdm(range(tot_samps2.shape[1])), init_func=init)  #range(tot_samps.shape[1])
plt.show()
vpath = check_path(FigPath,'version_{:d}'.format(0))
vname =  'DecodedVideo_{}_upsampled{:d}_smoothed{:d}_{}.mp4'.format(model_type,sf, win_size,cond)
writervideo = FFMpegWriter(fps=10) 
ani.save(os.path.join(vpath,vname), writer=writervideo)
print('DONE!!!')

In [ ]:
train_scores = train_scores.reshape((train_vid.shape[-2],train_vid.shape[-1]))
test_scores = test_scores.reshape((test_vid.shape[-2],test_vid.shape[-1]))
fig, axs = plt.subplots(2,1,figsize=(10,10))
im1 = axs[0].imshow(train_scores, vmin=0, vmax=.55)
axs[0].set_title('Train Correlation Map')
add_colorbar(im1)
im2 = axs[1].imshow(test_scores, vmin=0, vmax=.55)
axs[1].set_title('Test Correlation Map')
add_colorbar(im2)
plt.tight_layout()
# fig.savefig(FigPath/'DecodingScores_{}.png'.format(model_type),bbox_inches='tight',transparent=False, facecolor='w')

In [ ]:
t = 1040
dt = 20
comb = np.concatenate((np.concatenate((pred_test[t:t+dt,:,:], test_vid[t:t+dt,:,:]),axis=1)),axis=1)
fig, ax = plt.subplots(1,figsize=(25,20))
ax.imshow(comb)

Decoding wieghts of visual vs, decoding weights of movement and dot product overlap. 

# Pytorch

In [ ]:
# train_roll = train_roll/np.max(train_roll)
# train_roll -=train_roll[0]
# train_pitch = train_pitch/np.max(train_pitch)
# train_pitch -=train_pitch[0]
# test_roll = test_roll/np.max(test_roll)
# test_roll -=test_roll[0]
# test_pitch = test_pitch/np.max(test_pitch)
# test_pitch -=test_pitch[0]

# Y_train = torch.from_numpy(np.stack((train_roll, train_pitch),axis=1)).float()
# Y_test  = torch.from_numpy(np.stack((test_roll, test_pitch),axis=1)).float()

Y_train = torch.from_numpy(train_roll[:,np.newaxis]).float() #train_vid.reshape(train_vid.shape[0],-1)).float()#[:,10:11] # np.stack((train_roll, train_pitch),axis=1) # 
Y_test = torch.from_numpy(test_roll[:,np.newaxis]).float() #test_vid.reshape(test_vid.shape[0],-1)).float()#[:,10:11] # np.stack((test_roll, test_pitch),axis=1) # 

In [ ]:
class DecodingDataset(Dataset):
    def __init__(self, data, output, N_fm, transform=None):
        
        self.data = data
        self.output = output
        self.transform = transform
        self.N_fm = N_fm

    def __len__(self):
        return(self.data.shape[0])
    
    def __getitem__(self,idx):
        if idx < self.N_fm:
            idx = self.N_fm
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        sample = torch.from_numpy(self.data[idx-self.N_fm:idx]).float()
        gt = torch.from_numpy(self.output[idx-self.N_fm:idx,:]).float()
        return sample.view(-1), gt.view(-1)
    
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print            
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0
    
    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [ ]:
N_fm=1
batch_size = 1024
in_neurons = train_nsp.shape[1]*N_fm
out_neurons = 2048
out_dims = Y_train.shape[-1]*N_fm
NEpochs = 500
# train_dataset = DecodingDataset(train_nsp, np.stack((train_roll, train_pitch),axis=1), N_fm=N_fm)
# test_dataset = DecodingDataset(test_nsp, np.stack((test_roll, test_pitch),axis=1), N_fm=N_fm)
train_dataset = TensorDataset(torch.from_numpy(train_nsp).float(),Y_train)
test_dataset  = TensorDataset(torch.from_numpy(test_nsp).float(),Y_test)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=False)


In [ ]:
model = nn.Sequential(nn.Linear(in_neurons,out_neurons),
                      nn.ReLU(),
                      nn.Linear(out_neurons,out_neurons),
                      nn.ReLU(),
                      nn.Linear(out_neurons,out_dims)).to(device)
optimizer = optim.AdamW(params=model.parameters(), lr=.0001)
criteria = nn.MSELoss()
early_stopping = EarlyStopping(path=save_dir/'checkpoint.pt')

In [ ]:
tot_loss = []
test_tot_loss = []
for epoch in tqdm(range(NEpochs)):
    epoch_loss = []
    for batch, y in train_dataloader:
        pred = model(batch.to(device))
        loss = criteria(pred.to(device),y.to(device))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss.append(loss.item())
    tot_loss.append(np.mean(epoch_loss))
    
    with torch.no_grad():
        test_epoch_loss = []
        for batch, y in test_dataloader:
            pred = model(batch.to(device))
            loss = criteria(pred.to(device),y.to(device))
            test_epoch_loss.append(loss.item())
        test_tot_loss.append(np.mean(test_epoch_loss))
    early_stopping(np.mean(test_epoch_loss), model)
    if early_stopping.early_stop == True:
        print('Stopped Early!')
        break
    print('Epoch:', epoch, 'Epoch_Loss_Avg: ', np.mean(epoch_loss), 'Test_Epoch_Loss_Avg: ', np.mean(test_epoch_loss))

In [ ]:
wind = np.arange(0,1000)
fig, ax = plt.subplots(2,1,figsize=(20,10))
ax[0].plot(Y_train[wind,0],'b-', label='roll')
ax[0].plot(pred[wind,0].cpu().detach(),'r-', label='pred_roll')
ax[1].plot(Y_train[wind,1],'b-', label='pitch')
ax[1].plot(pred[wind,1].cpu().detach(),'r-', label='pred_pitch')
ax[0].legend()
ax[1].legend()

In [ ]:
with torch.no_grad():
    predt = []
    for batch, y in test_dataloader:
        pred = model(batch.to(device))
        predt.append(pred.cpu().numpy())
    predt = np.concatenate(predt,axis=0)

In [ ]:
wind = np.arange(0,1000)
fig, ax = plt.subplots(2,1,figsize=(20,10))
ax[0].plot(Y_test[wind,0],'b-', label='roll')
ax[0].plot(predt[wind,0],'r-', label='pred_roll')
ax[1].plot(Y_test[wind,1],'b-', label='pitch')
ax[1].plot(predt[wind,1],'r-', label='pred_pitch')